<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Astro_Flask_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok --quiet
!pip install flask yfinance
!pip install flatlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, render_template
from pyngrok import ngrok, conf
import os
import yfinance as yf
from datetime import datetime

# Вставь свой authtoken
conf.get_default().auth_token = "1pmyR9yMjWGv3Hv3Or9ghr8zMY7_6ncV13tdrFrbqQrHenKJR"

# Укажи путь к шаблонам на Google Drive
template_dir = "/content/drive/MyDrive/timefactor/templates"


app = Flask(__name__, template_folder=template_dir)

@app.route('/get_ohlc', methods=['POST'])
def get_ohlc():
    symbol = request.json.get('symbol', 'BTC-USD')
    interval = request.json.get('interval', '1d')
    start = request.json.get('start')
    end = request.json.get('end')

    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)

    ohlc = []
    for i, row in df.iterrows():
        timestamp = int(datetime.timestamp(row['Date']) * 1000)
        ohlc.append([timestamp, row['Open'], row['High'], row['Low'], row['Close']])

    return jsonify({'status': 'success', 'data': ohlc})

def index():
    return render_template("index.html")

# Проброс порта
public_url = ngrok.connect(5000)
print("🚀 Публичный адрес:", public_url)

# Запуск Flask-сервера
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from flask import Flask, render_template, request, jsonify
import yfinance as yf
from datetime import datetime, timedelta
import time
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib import const
from pyngrok import ngrok, conf

# Вставь свой authtoken
conf.get_default().auth_token = "1pmyR9yMjWGv3Hv3Or9ghr8zMY7_6ncV13tdrFrbqQrHenKJR"

# Укажи путь к шаблонам на Google Drive
template_dir = "/content/drive/MyDrive/timefactor/templates"

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_ohlc', methods=['POST'])
def get_ohlc():
    data = request.get_json()
    symbol = data.get('symbol', 'BTC-USD')
    interval = data.get('interval', '1d')
    start = data.get('start', '2021-07-05')
    end = data.get('end', datetime.today().strftime('%Y-%m-%d'))

    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)

    ohlc = []
    for _, row in df.iterrows():
        timestamp = int(time.mktime(row['Date'].timetuple()) * 1000)
        ohlc.append([timestamp, row['Open'], row['High'], row['Low'], row['Close']])

    return jsonify({'status': 'success', 'data': ohlc})

@app.route('/get_planet_longitude', methods=['POST'])
def get_planet_longitude():
    data = request.get_json()
    start_str = data.get('start')
    end_str = data.get('end')
    planet_name = data.get('planet', 'MOON').upper()
    system = data.get('system', 'geo')
    show_aspect_lines = data.get('aspect_lines', False)
    orbis = float(data.get('orbis', 2.0))
    aspect_angles = data.get('angles', [0, 60, 90, 120, 180])
    comparison_planets = data.get('compare', ['SUN', 'MERCURY', 'VENUS', 'MARS', 'JUPITER', 'SATURN'])

    start_date = datetime.strptime(start_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_str, "%Y-%m-%d")

    pos = GeoPos("0", "51")  # Лондон
    delta = timedelta(days=1)

    results = []
    aspects = []

    while start_date <= end_date:
        dt = Datetime(start_date.strftime("%Y-%m-%d"), "00:00", "+00:00")
        chart = Chart(dt, pos, ID=system.upper())
        planet = chart.get(planet_name)
        timestamp = int(time.mktime(start_date.timetuple()) * 1000)
        results.append([timestamp, round(planet.lon, 2)])

        if show_aspect_lines:
            for other_name in comparison_planets:
                if other_name.upper() != planet_name:
                    try:
                        other = chart.get(other_name.upper())
                        diff = abs(planet.lon - other.lon) % 360
                        for angle in aspect_angles:
                            if abs(diff - angle) <= orbis:
                                aspects.append({
                                    'timestamp': timestamp,
                                    'angle': angle,
                                    'with': other_name.upper(),
                                    'exact': round(diff, 2)
                                })
                    except Exception:
                        continue

        start_date += delta

    return jsonify({'status': 'success', 'data': results, 'aspects': aspects})

# Проброс порта
public_url = ngrok.connect(5000)
print("🚀 Публичный адрес:", public_url)

if __name__ == '__main__':
    app.run(port=5000)

🚀 Публичный адрес: NgrokTunnel: "https://71a5-34-73-216-142.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
          